In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pysal as ps
from datetime import datetime
from datetime import timedelta

%matplotlib inline

/Users/brianconnor/anaconda3/lib/python3.6/site-packages/pysal/__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://pysal.org/about
  ), VisibleDeprecationWarning)


## Project Goals:
- Analyze the data to see how weather and location relate to the prevalence of WNV
- Predict the most likely location of mosquitos carrying WNV
- Make recommendations on pesticide sprays based on cost, area, and efficacy


In [2]:
weather_csv = "./data/weather.csv"
weather = pd.read_csv(weather_csv)

spray_csv = "./data/spray.csv"
spray = pd.read_csv(spray_csv)

train_csv = "./data/train.csv"
train = pd.read_csv(train_csv)

test_csv = "./data/test.csv"
test = pd.read_csv(test_csv)

train_weather_csv = "./data/train_weather.csv"
tw = pd.read_csv(train_weather_csv)

train_weather_per_station = "./data/train_weather_per_station.csv"
twps = train_weather_per_station = pd.read_csv(train_weather_per_station)

In [3]:
pd.set_option('display.max_columns', 37)

### Things to check out:
- Does spraying have any effect on mosquito prevalence
- Does spraying have any effect on WNV prevalence
- Does weather have any effect on skeeters/WNV

- What areas have the most wnv/mosquitoes

In [4]:
weather.head()
# column info:
# Tmax = max temperature
# Tmin = min temperature
# Tavg = average temperature
# depart = departure from normal
# dewpoint = avg temp where dew forms
# wetbulb = avg temp measured by a wet bulb thermometer
# heat = heating season (starting in july). I dont understand the number
# cool = cooling season (starting in january). still dont understand the number
# sunrise = calculated time of sunrise
# sunset = calculated time of sunset
# codesum = significant weather codes
# depth = amount of snow/ice on the ground (T is trace)
# water1 = water equivalent of "depth" column
# snowfall = snowfall in inches
# preciptotsl = total water from rain and melted snow
# stnpressure = avg pressure in inches of mercury (inches?? guess they use those for the weather)
# sealevel = pressure at sea level
# resultspeed = wind speed in mph
# resultdir = wind direction
# avgspeed = average speed

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,-,-,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,-,-,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [5]:
train[train["WnvPresent"]==1]

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
553,2007-07-18,"3800 East 115th Street, Chicago, IL 60617, USA",CULEX PIPIENS/RESTUANS,38,E 115TH ST,T215,"3800 E 115TH ST, Chicago, IL",41.686398,-87.531635,8,50,1
603,2007-07-25,"South Doty Avenue, Chicago, IL, USA",CULEX PIPIENS/RESTUANS,12,S DOTY AVE,T115,"1200 S DOTY AVE, Chicago, IL",41.673408,-87.599862,5,50,1
611,2007-07-25,"South Doty Avenue, Chicago, IL, USA",CULEX PIPIENS/RESTUANS,12,S DOTY AVE,T115,"1200 S DOTY AVE, Chicago, IL",41.673408,-87.599862,5,50,1
618,2007-07-25,"South Doty Avenue, Chicago, IL, USA",CULEX PIPIENS,12,S DOTY AVE,T115,"1200 S DOTY AVE, Chicago, IL",41.673408,-87.599862,5,50,1
660,2007-07-25,"South Doty Avenue, Chicago, IL, USA",CULEX PIPIENS,12,S DOTY AVE,T115,"1200 S DOTY AVE, Chicago, IL",41.673408,-87.599862,5,50,1
661,2007-07-25,"South Doty Avenue, Chicago, IL, USA",CULEX PIPIENS,12,S DOTY AVE,T115,"1200 S DOTY AVE, Chicago, IL",41.673408,-87.599862,5,50,1
777,2007-08-01,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,50,1
778,2007-08-01,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,21,1
784,2007-08-01,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,5,1
812,2007-08-01,"2200 West 113th Street, Chicago, IL 60643, USA",CULEX PIPIENS/RESTUANS,22,W 113TH ST,T086,"2200 W 113TH ST, Chicago, IL",41.688324,-87.676709,8,33,1


In [6]:
# TW is a dataframe of the train set combined with the weather set that Emma G created
tw['Date'] = pd.to_datetime(tw.Date)

In [10]:
tw

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,CULEX PIPIENS,CULEX PIPIENS/RESTUANS,CULEX RESTUANS,CULEX SALINARIUS,CULEX TARSALIS,CULEX TERRITANS,AvgSpeed,Cool,Depart,Depth,DewPoint,Heat,PrecipTotal,ResultDir,ResultSpeed,SeaLevel,SnowFall,Station,StnPressure,Sunrise,Sunset,Tavg,Tmax,Tmin,WetBulb
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,0,1,0,0,0,0,6.95,10.5,10.0,0.0,58.5,0.0,0.000,17.0,5.80,30.100,0.0,1.5,29.415,421.0,1917.0,75.5,88.0,62.5,65.5
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,0,0,1,0,0,0,6.95,10.5,10.0,0.0,58.5,0.0,0.000,17.0,5.80,30.100,0.0,1.5,29.415,421.0,1917.0,75.5,88.0,62.5,65.5
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0,0,0,1,0,0,0,6.95,10.5,10.0,0.0,58.5,0.0,0.000,17.0,5.80,30.100,0.0,1.5,29.415,421.0,1917.0,75.5,88.0,62.5,65.5
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0,0,1,0,0,0,0,6.95,10.5,10.0,0.0,58.5,0.0,0.000,17.0,5.80,30.100,0.0,1.5,29.415,421.0,1917.0,75.5,88.0,62.5,65.5
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0,0,0,1,0,0,0,6.95,10.5,10.0,0.0,58.5,0.0,0.000,17.0,5.80,30.100,0.0,1.5,29.415,421.0,1917.0,75.5,88.0,62.5,65.5
5,2007-05-29,"1500 West Webster Avenue, Chicago, IL 60614, USA",CULEX RESTUANS,15,W WEBSTER AVE,T045,"1500 W WEBSTER AVE, Chicago, IL",41.921600,-87.666455,8,2,0,0,0,1,0,0,0,6.95,10.5,10.0,0.0,58.5,0.0,0.000,17.0,5.80,30.100,0.0,1.5,29.415,421.0,1917.0,75.5,88.0,62.5,65.5
6,2007-05-29,"2500 West Grand Avenue, Chicago, IL 60654, USA",CULEX RESTUANS,25,W GRAND AVE,T046,"2500 W GRAND AVE, Chicago, IL",41.891118,-87.654491,8,1,0,0,0,1,0,0,0,6.95,10.5,10.0,0.0,58.5,0.0,0.000,17.0,5.80,30.100,0.0,1.5,29.415,421.0,1917.0,75.5,88.0,62.5,65.5
7,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,11,W ROOSEVELT,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,1,0,0,1,0,0,0,0,6.95,10.5,10.0,0.0,58.5,0.0,0.000,17.0,5.80,30.100,0.0,1.5,29.415,421.0,1917.0,75.5,88.0,62.5,65.5
8,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX RESTUANS,11,W ROOSEVELT,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,2,0,0,0,1,0,0,0,6.95,10.5,10.0,0.0,58.5,0.0,0.000,17.0,5.80,30.100,0.0,1.5,29.415,421.0,1917.0,75.5,88.0,62.5,65.5
9,2007-05-29,"1100 West Chicago Avenue, Chicago, IL 60642, USA",CULEX RESTUANS,11,W CHICAGO,T049,"1100 W CHICAGO, Chicago, IL",41.896282,-87.655232,8,1,0,0,0,1,0,0,0,6.95,10.5,10.0,0.0,58.5,0.0,0.000,17.0,5.80,30.100,0.0,1.5,29.415,421.0,1917.0,75.5,88.0,62.5,65.5


### Mosquito Consolidation
In the context of what we're looking for (is WNV present **here**), I think it's important to merge the data entries the count of mosquitos was more than 50, and so a new data entry was made. Some of these entries indicate west nile and some don't, however, this would be inaccurate when fed into a model

Say there were 50 mosquitos with WNV present and 50 mosquitos with WNV not present (same species) in a given spot at a given time. If you considered the data points separately, it's like giving the fact that WNV is present only half the weight in a model it should have. In truth (in that given area at that given time) WNV IS present, even though some batches of mosquitos indicate it and some don't. So mosquito counts should be combined to show 100 + WNV present because that would be the most accurate representation of the area

In [ ]:
# first I'm summing every column on the date and location just to get the mosquito totals
new = train.groupby(by=["Date","AddressNumberAndStreet", "Species"], sort=False).sum()

In [ ]:
new.head(20)

# at this point the only thing that's important is the number of mosquitos and corresponding species/address/date 

In [ ]:
# now i'm removing all duplicates on date and location from the *main* dataframe

# I'm assuming that for each location at a given time, everything will be recorded equally except mosquito
# count and WNV presence

train.drop_duplicates(subset=["Date", "AddressNumberAndStreet", "Species"], keep="first", inplace=True)
train.head()

In [ ]:
new.head()

In [ ]:
# only way I can figure out how to add the new count to the train dataframe is by dropping and adding

In [ ]:
train.drop(labels="NumMosquitos", axis=1, inplace=True)

In [ ]:
mosqcountlist = new.NumMosquitos.tolist()

In [ ]:
train["NumMosquitos"]= mosqcountlist

In [ ]:
# I did it. I'm the best
train.NumMosquitos.max()

#### now I'm going to try to do this to Emma's combined dataframe by "un-dummying" the species columns 

In [ ]:
tw.head()

In [ ]:
# making a new dataframe that only includes the dummy columns
species = ["CULEX PIPIENS", "CULEX PIPIENS/RESTUANS", "CULEX RESTUANS", "CULEX SALINARIUS", "CULEX TARSALIS", "CULEX TERRITANS"]
specdf = tw[species]

# this command I just found on google. It does something like returning the index of the entry. So if axis is set
# to 1, the column name is the index
specdf = specdf.idxmax(axis=1)
specdf = pd.DataFrame(data=specdf, columns=["Species"])
specdf.head()

In [ ]:
# dropping the dummies and replacing them with the species column
twps.drop(labels=species, axis=1, inplace=True)
twps["Species"] = specdf["Species"]

In [ ]:
twps.head()
# done

In [ ]:
# now doing what I did before to combined mosquito counts

new_2 = twps.groupby(by=["Date","AddressNumberAndStreet", "Species"], sort=False).sum()

In [11]:
# im going to redo the cleaning in a different notebok because I just made a new csv with different info